Import Azure Open AI Libraries

In [5]:
%pip install openai

import os
import openai
from pyspark.sql.functions import col, concat_ws, lit, collect_list
from pyspark.sql import SparkSession
import os

from openai import AzureOpenAI

print("Libraries imported successfully.")

StatementMeta(, 9aeee145-3471-489f-b850-817f6adf7f6e, 18, Finished, Available, Finished)


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Libraries imported successfully.



Configure API Keys

In [9]:
# --- CONFIGURATION ---


# --- CONFIGURATION ---

# --- Azure OpenAI Configuration ---
# Make sure AZURE_OPENAI_ENDPOINT and AZURE_OPENAI_KEY are set as environment variables
endpoint = "API_BASE"

model_name = "gpt-4o"

deployment = "DEPLOYMENT_NAME"
api_key = "API_KEY"
api_version = "2024-12-01-preview"



# Initialize the AzureOpenAI client (new SDK)
# This client object will be used for making API calls
try:
    client = AzureOpenAI(
        api_version=api_version,
        azure_endpoint=endpoint,
        api_key=api_key,
    )
    print("AzureOpenAI client initialized successfully.")
except Exception as e:
    print(f"Error initializing AzureOpenAI client: {e}")
    raise e
csv_file_path = "abfss://8d3e063b-5552-4052-b919-22ef780dcf63@onelake.dfs.fabric.microsoft.com/ee0b2da1-5ca6-4b9b-8bfb-49e6d3e66ee1/Tables/dbo/SPICEdata csv"

# --- Data Sampling Configuration ---
# Number of rows to include as context in the prompt (be mindful of token limits)
sample_rows_for_context = 20

#

StatementMeta(, 9aeee145-3471-489f-b850-817f6adf7f6e, 23, Finished, Available, Finished)

AzureOpenAI client initialized successfully.


Load Data from Lakehouse

In [12]:
# Load the CSV file into a Spark DataFrame
# Adjust options as needed (e.g., delimiter, encoding)
try:
    df = df = spark.sql("SELECT * FROM model_data.dbo.`SPICEdata csv` LIMIT 1000")
    print(f"Successfully loaded data from: {csv_file_path}")
    print("Schema:")
    df.printSchema()
    print("Sample Data:")
    display(df.limit(20)) # Use display() for richer output in Fabric notebooks
except Exception as e:
    print(f"Error loading CSV file from path: {csv_file_path}")
    print(f"Please ensure the path is correct and the Lakehouse is attached.")
    raise e

StatementMeta(, 9aeee145-3471-489f-b850-817f6adf7f6e, 26, Finished, Available, Finished)

Successfully loaded data from: abfss://8d3e063b-5552-4052-b919-22ef780dcf63@onelake.dfs.fabric.microsoft.com/ee0b2da1-5ca6-4b9b-8bfb-49e6d3e66ee1/Tables/dbo/SPICEdata csv
Schema:
root
 |-- Year: long (nullable = true)
 |-- Month: date (nullable = true)
 |-- Dept_Name: string (nullable = true)
 |-- Section_Name: string (nullable = true)
 |-- Family_name: string (nullable = true)
 |-- Sub_Family_Name: string (nullable = true)
 |-- Brand_Principle: string (nullable = true)
 |-- Brand_Name: string (nullable = true)
 |-- SupplierName: string (nullable = true)
 |-- Item_Code: long (nullable = true)
 |-- Item_Bar_Code: long (nullable = true)
 |-- Item_Name: string (nullable = true)
 |-- BRAND: string (nullable = true)
 |-- TYPE: string (nullable = true)
 |-- VARIANTS_2: string (nullable = true)
 |-- CATEGORY: string (nullable = true)
 |-- SUPER_CATEGORY: string (nullable = true)
 |-- PACKAGING: string (nullable = true)
 |-- PRICE: double (nullable = true)
 |-- Storename: string (nullable = tr

SynapseWidget(Synapse.DataFrame, 593d1a6f-5037-4c5f-bf34-9b8ef1cf8f05)

Prepare for LLM ingestion

In [13]:
# Select a sample of the data to provide context to the LLM
# Convert the sample to a string format suitable for the prompt

if sample_rows_for_context > 0:
    sample_df = df.limit(sample_rows_for_context)
    # Convert the sample DataFrame rows to a single string representation
    # Here, we format it like a simple text table or list
    sample_data_list = sample_df.collect() # Collect rows to the driver (only for small samples!)

    if sample_data_list:
        header = ", ".join(df.columns)
        rows_str = [", ".join(map(str, row)) for row in sample_data_list]
        data_context_string = f"Column Headers: {header}\nSample Rows:\n" + "\n".join(rows_str)
        print("Sample data prepared for context string.")
        # Optional: Print the context string to see what's sent (can be long)
        # print("\n--- Data Context String ---")
        # print(data_context_string)
        # print("--------------------------\n")
    else:
        data_context_string = "No sample data available to provide."
        print("Warning: No data rows found to create context string.")
else:
     data_context_string = "No specific data sample provided for context. You know the schema."
     print("Data sampling is disabled (sample_rows_for_context=0). Only schema info might be implicitly used.")


# Also good to provide schema info explicitly
schema_string = "\n".join([f"- {field.name}: {field.dataType.simpleString()}" for field in df.schema.fields])
full_context_description = f"You are analyzing data with the following schema:\n{schema_string}\n\nHere is a small sample of the data:\n{data_context_string}"

StatementMeta(, 9aeee145-3471-489f-b850-817f6adf7f6e, 27, Finished, Available, Finished)

Sample data prepared for context string.


Azure Open AI Queries

In [19]:
def query_openai_sync(aoai_client, question, data_context, deployment_name):
    """
    Sends a question to Azure OpenAI (synchronously) along with data context
    using the modern SDK client.

    Args:
        aoai_client (AzureOpenAI): The initialized AzureOpenAI client.
        question (str): The user's question about the data.
        data_context (str): A string containing schema and/or sample data.
        deployment_name (str): The Azure OpenAI deployment name (model).

    Returns:
        str: The content of the AI's response, or an error message.
    """
    try:
        messages = [
            {
                "role": "system",
                "content": f"You are an AI assistant helping analyze data. Use the provided schema and sample data to answer the user's question accurately. If the data doesn't contain the answer, say so clearly.\n\nData Context:\n{data_context}"
            },
            {
                "role": "user",
                "content": question
            }
        ]

        print(f"\n--- Sending prompt to deployment '{deployment_name}' (Sync) ---")

        response = aoai_client.chat.completions.create(
            model=deployment_name, # Use 'model' parameter in new SDK
            messages=messages,
            temperature=0.7,
            max_tokens=800
        )

        print("--- Received synchronous response ---")
        # Access response content using object attributes
        return response.choices[0].message.content.strip()

    except openai.APIAuthenticationError as e:
        print(f"OpenAI Authentication Error: {e}")
        return f"Error: Authentication failed. Check your API key and endpoint configuration. ({e})"
    except openai.APIConnectionError as e:
        print(f"OpenAI Connection Error: {e}")
        return f"Error: Could not connect to the Azure OpenAI endpoint. Check the endpoint URL and network connectivity. ({e})"
    except openai.RateLimitError as e:
        print(f"OpenAI Rate Limit Error: {e}")
        return f"Error: Rate limit exceeded. Please wait and try again or check your quota. ({e})"
    except openai.APIError as e: # Catch other API-related errors
        print(f"OpenAI API Error: {e.status_code} - {e.response}")
        return f"Error: An API error occurred. Status Code: {e.status_code}. ({e})"
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return f"Error: An unexpected error occurred during the OpenAI request. ({e})"

StatementMeta(, 9aeee145-3471-489f-b850-817f6adf7f6e, 33, Finished, Available, Finished)

AI response

In [21]:
# --- Ask your question here! ---
user_question = "Based on the sample data, what seems to be the main topic or theme?"
# Example questions (adapt to your data):
# user_question = "Summarize the 'FeedbackComments' from the sample data."
user_question = "Identify any potential optimization points according to the sales trends"

# --- Call the synchronous function and display the result ---
# Pass the initialized client object
ai_response = query_openai_sync(client, user_question, full_context_description, deployment)

print(f"\nQuestion: {user_question}")
print(f"\nAI Response (Sync):\n---------------------\n{ai_response}\n---------------------")

StatementMeta(, 9aeee145-3471-489f-b850-817f6adf7f6e, 35, Finished, Available, Finished)


--- Sending prompt to deployment 'gpt-4o-2' (Sync) ---
--- Received synchronous response ---

Question: Identify any potential optimization points according to the sales trends

AI Response (Sync):
---------------------
The provided data contains a record of sales quantities (`Qty`) for different items across multiple stores (`Storename`). Based on this data, some potential optimization points can be identified:

### 1. **Focus on High-Selling Stores**
   - Some stores have significantly higher sales quantities of a particular product. For example:
     - `8124 - KSRT` sold **204.0 units** of "TROPICAL HEAT ONION POWDER 100G."
     - `3110 - KRIV` sold **203.0 units** of the same item.
   - These stores could be prioritized for stock replenishment or promotional campaigns to maintain and enhance sales levels.

### 2. **Identify and Address Low-Performing Stores**
   - Some stores have very low sales quantities for the same product:
     - `8111 - KXKI2` sold only **2.0 units**.
     -